In [17]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy import stats 

In [18]:
# Preparing dataset from MusicBrainz
musicbrainz= pd.read_csv('musicbrainz_artist.txt', sep='|')
musicbrainz= musicbrainz.drop(columns=['ID'])
musicbrainz= musicbrainz[['Name','Gender','Genre','Type','Birthday']]
musicbrainz['Name']= musicbrainz['Name'].str.lower()
musicbrainz = musicbrainz[musicbrainz['Type'] == ' Person']

# Filtered by Electronic Music Genre 
genre_list = open('genrelist.txt','r')
lines = genre_list.readlines()
lines = lines[0]
genrelist = lines.split(',')
genre_list.close()

musicbrainz= musicbrainz[musicbrainz['Genre'].str.contains('|'.join(genrelist))]
#musicbrainz.head()

#len(musicbrainz)

In [4]:
# Preparing DataSet for Songkick all gigs
songkick= pd.read_csv('songkick_all_gigs_extended_version.tsv', sep='	')
songkick= songkick.drop(columns=['ID'])
songkick['Year'] = pd.DatetimeIndex(songkick['Date']).year
songkick= songkick.drop(columns=['Date','City','Venue'])
GigCount = songkick.groupby(['Name']).count()


GigCount = GigCount.drop(columns=['Country'])
GigCount = GigCount.rename(index=str, columns={'Year': 'GigCount'})

In [5]:
# Adding all countries for each artists
Countries = songkick.groupby('Name')['Country'].apply(','.join)
Countries = Countries.to_frame()

In [6]:
songkick['Year'] = songkick['Year'].apply(str)
ConcertYears = songkick.groupby('Name')['Year'].apply(','.join)
ConcertYears = ConcertYears.to_frame()

In [7]:
songkick = pd.merge(songkick, Countries, on='Name',how='inner')
#songkick.head()

In [8]:
songkick = pd.merge(songkick, GigCount, on='Name',how='inner')
#songkick.head()

In [9]:
songkick = pd.merge(songkick, ConcertYears, on='Name',how='inner')
#songkick.head()

In [10]:
songkick= songkick.drop(columns=['Country_x','Year_x'])  #'index' remove 
songkick= songkick.rename(index=str, columns={'Country_y': 'Countries', 'Year_y':'ConcertYears'})
songkick= songkick.drop_duplicates()
songkick = songkick.reset_index()
#songkick.head()

In [11]:
# Merging musicbrainz and songkick datasets on the basis of 'Name'

merge1= pd.merge(songkick, musicbrainz, on='Name',how='inner')
merge1= merge1.drop(columns=['index','Type','Genre'])
merge1.to_csv('Merge1.csv', index= True, sep= ',') #saved to merge1.csv
#merge1.head()

# giving the numbers of total gigs
# sum(merge1.GigCount)


In [12]:
# sum(merge1.GigCount)

In [13]:
# Preparing dataset for releases merged dataset
release= pd.read_csv('releases_merged.csv', sep=',')
release=release.drop(columns=['artist_id','release_id','label_name','Genre', 'Release_Name'])
release.rename(columns={'Year':'Release_Year'}, inplace=True)

release['Release_Year'] = release['Release_Year'].apply(str)
releaseYear = release.groupby('Name')['Release_Year'].apply(','.join)
releaseYear = releaseYear.to_frame()

AlbumCount = release.groupby(['Name']).count()
AlbumCount = AlbumCount.drop(columns=['Release_Year'])
AlbumCount = AlbumCount.rename(index=str, columns={'Price': 'AlbumCount'})

avgPrice = release.groupby('Name')['Price'].mean()
avgPrice = avgPrice.to_frame()

C:\Users\Mishu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
# Making new release dataset
release = pd.merge(release, releaseYear, on='Name',how='inner')
release = pd.merge(release, AlbumCount, on='Name',how='inner')
release = pd.merge(release, avgPrice, on='Name',how='inner')

# Dropping Price_x, Release_Year_x
release = release.drop(columns=['Price_x', 'Release_Year_x'])
release = release.drop_duplicates()
release = release.rename(index=str, columns={'Release_Year_y': 'Release_Years', 'Price_y':'AvgPrice'})

#release.head()

In [15]:
# Making Final Merged Dataset
finalMerge= pd.merge(merge1, release, on='Name', how='inner')
finalMerge.to_csv('FinalMerge.csv', index= True, sep= ',')

# Male-Female Separate Dataset
#male = finalMerge[finalMerge['Gender'] == ' Male']
#female = finalMerge[finalMerge['Gender'] == ' Female']

In [16]:
finalMerge.head()

,Name,Countries,GigCount,ConcertYears,Gender,Birthday,Release_Years,AlbumCount,AvgPrice
0,oneohtrix point never,"uk,us,croatia,us,us,spain,us,croatia,belgium,c...",262,"2010,2016,2015,2016,2016,2018,2019,2011,2010,2...",Male,25/7/1982,"2018,2018,2018,2018,2018,2016,2013,2017,2017,2...",13,5.840000
1,james ferraro,"us,us,croatia,croatia,croatia,switzerland,us,u...",78,"2010,2011,2018,2013,2013,2014,2010,2010,2015,2...",Male,7/11/1986,"2013,2012,2013,2015,2013",5,13.104000
2,scuba,"uk,croatia,mexico,uk,uk,us,us,us,france,france...",404,"2016,2014,2016,2012,2013,2014,2014,2014,2014,2...",Male,\N/\N/\N,"2018,2018,2017,2015,2013,2011,2010,2018,2009,2...",69,6.273913
3,grimes,"us,us,us,us,us,us,uk,canada,croatia,croatia,fi...",295,"2016,2016,2016,2016,2013,2013,2012,2016,2011,2...",Female,17/3/1988,"2014,2012,2012,2015,2015,2011,2010,2010,2011",9,19.520000
4,ryan hemsworth,"us,us,us,uk,germany,us,us,denmark,croatia,us,a...",178,"2016,2014,2013,2014,2014,2013,2014,2014,2018,2...",Male,\N/\N/\N,"2018,2013",2,3.640000


In [ ]:
# end